# Lab 3: Advanced Agent Patterns

**Module 3 - Advanced Agent Development**

| Duration | Difficulty | Framework | Exercises |
|----------|------------|-----------|----------|
| 120 min | Advanced | LangChain | 3 |

## Learning Objectives

- Implement Chain-of-Thought reasoning
- Build a Plan-and-Execute agent
- Create a self-reflecting agent with critique

## Setup

In [ ]:
# !pip install langchain langchain-openai

In [ ]:
import os
import json
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import HumanMessage, SystemMessage

os.environ["OPENAI_API_KEY"] = "your-api-key-here"

llm = ChatOpenAI(model="gpt-4", temperature=0.3)

---

## Exercise 1: Implement Chain-of-Thought Prompting

Create a CoT prompt template that encourages step-by-step reasoning.

**Your Task:** Design a prompt that guides the model through structured reasoning.

In [ ]:
def create_cot_prompt(question: str) -> str:
    """Create a Chain-of-Thought prompt."""
    # TODO: Design a prompt that encourages step-by-step reasoning
    # Include explicit instructions to:
    # 1. Identify what needs to be found
    # 2. Break down the problem
    # 3. Work through each part
    # 4. Combine results for final answer
    
    prompt = f"""
Question: {question}

Let's solve this step by step:

Step 1: [Identify what we need to find]
Step 2: [Break down the problem]
Step 3: [Work through each part]
Step 4: [Combine results]

Final Answer:
"""
    return prompt


def solve_with_cot(question: str) -> dict:
    """Solve a problem using Chain-of-Thought."""
    prompt = create_cot_prompt(question)
    
    # TODO: Call the LLM and parse the response
    response = None  # Your code here
    
    return {
        "question": question,
        "reasoning": response,
    }

In [ ]:
# Test problems
problems = [
    "A store has 45 apples. They sell 12 in the morning and receive a shipment of 30. How many apples do they have now?",
    "If a train travels at 60 mph for 2 hours, then 80 mph for 1.5 hours, what's the total distance?",
    "A rectangle has a perimeter of 24 cm. If the length is twice the width, what are the dimensions?"
]

for problem in problems:
    result = solve_with_cot(problem)
    print(f"\n{'='*60}")
    print(f"Problem: {problem}")
    print(f"\nReasoning:\n{result['reasoning']}")

---

## Exercise 2: Build a Plan-and-Execute Agent

Create an agent that first creates a plan, then executes each step.

**Your Task:** Implement the planning and execution phases.

In [ ]:
class PlanAndExecuteAgent:
    def __init__(self, llm):
        self.llm = llm
        self.plan = []
        self.results = []
    
    def create_plan(self, task: str) -> list:
        """Create a step-by-step plan for the task."""
        planning_prompt = f"""
You are a planning assistant. Create a detailed step-by-step plan for this task.
Return the plan as a numbered list (1., 2., 3., etc.).
Keep each step clear and actionable.

Task: {task}

Plan:
1."""
        
        # TODO: Generate plan and parse into list
        response = None  # Your code here
        
        # Parse numbered list into steps
        # Your parsing code here
        self.plan = []
        
        return self.plan
    
    def execute_step(self, step: str, context: str) -> str:
        """Execute a single step of the plan."""
        execution_prompt = f"""
Previous context: {context}

Current step to execute: {step}

Execute this step and provide a detailed result:
"""
        # TODO: Execute the step
        response = None  # Your code here
        return response
    
    def run(self, task: str) -> dict:
        """Run the full plan-and-execute cycle."""
        # Phase 1: Planning
        print("Creating plan...")
        plan = self.create_plan(task)
        print(f"Plan created with {len(plan)} steps")
        
        # Phase 2: Execution
        context = ""
        for i, step in enumerate(plan):
            print(f"\nExecuting step {i+1}: {step[:50]}...")
            result = self.execute_step(step, context)
            self.results.append({"step": step, "result": result})
            context += f"\nStep {i+1} result: {result}"
        
        return {
            "task": task,
            "plan": plan,
            "results": self.results
        }

In [ ]:
# Test the agent
# agent = PlanAndExecuteAgent(llm)
# result = agent.run("Research and summarize the key differences between BERT and GPT models")
# print(result)

---

## Exercise 3: Create a Self-Reflecting Agent

Build an agent that critiques and improves its own output.

**Your Task:** Implement the generate, critique, and improve cycle.

In [ ]:
class ReflectiveAgent:
    def __init__(self, llm, max_iterations=3):
        self.llm = llm
        self.max_iterations = max_iterations
    
    def generate(self, task: str) -> str:
        """Generate initial response."""
        prompt = f"Complete this task:\n{task}"
        # TODO: Generate response
        response = None  # Your code here
        return response
    
    def critique(self, task: str, response: str) -> dict:
        """Critique the response and suggest improvements."""
        critique_prompt = f"""
Task: {task}

Response to critique:
{response}

Analyze this response and provide:
1. Score (1-10)
2. Strengths
3. Weaknesses
4. Specific improvements needed

Format as JSON:
{{"score": X, "strengths": [...], "weaknesses": [...], "improvements": [...]}}
"""
        # TODO: Generate critique and parse JSON
        result = None  # Your code here
        
        try:
            return json.loads(result)
        except:
            return {"score": 5, "improvements": ["Could not parse critique"]}
    
    def improve(self, task: str, response: str, critique: dict) -> str:
        """Improve response based on critique."""
        improve_prompt = f"""
Task: {task}

Previous response:
{response}

Critique feedback:
- Weaknesses: {critique.get('weaknesses', [])}
- Improvements needed: {critique.get('improvements', [])}

Write an improved response addressing the feedback:
"""
        # TODO: Generate improved response
        result = None  # Your code here
        return result
    
    def run(self, task: str) -> dict:
        """Run the reflection loop."""
        response = self.generate(task)
        history = [{"iteration": 0, "response": response}]
        
        for i in range(self.max_iterations):
            critique = self.critique(task, response)
            print(f"Iteration {i+1} - Score: {critique.get('score', 'N/A')}")
            
            if critique.get('score', 0) >= 8:
                print("Quality threshold met!")
                break
            
            response = self.improve(task, response, critique)
            history.append({
                "iteration": i + 1,
                "critique": critique,
                "response": response
            })
        
        return {"final_response": response, "history": history}

In [ ]:
# Test the reflective agent
# agent = ReflectiveAgent(llm)
# result = agent.run("Write a concise explanation of how neural networks learn")
# print(f"\nFinal Response:\n{result['final_response']}")

---

## Checkpoint

Congratulations! You've completed Lab 3. You should now understand:

- How Chain-of-Thought improves reasoning on complex problems
- How Plan-and-Execute separates strategy from execution
- How self-reflection enables iterative improvement

**Next:** Lab 4 - RAG Pipeline Implementation